<a href="https://colab.research.google.com/github/nidhimenonn/Thesis/blob/master/NEW_62_round.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
from zipfile import ZipFile
!pip install gspread-pandas
from gspread_pandas import Spread, Client

In [20]:
# Mounting the Drive

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Installing and Aunthenticating PyDrive

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# **Nutritional Value:**

In [22]:
# NUTRITIONAL VALUE PART 1

# link to the dta file
link = 'https://drive.google.com/open?id=1aiSfPuDDbTE9D9wtrq5GEFgl9jXEZoeM'

# verify that you have everything after '='
fluff, id = link.split('=')
print (id) 

# store dataset in pandas
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('nss61list.dta')  

#read the file
df1 = pd.read_stata('nss61list.dta')
df1.columns


df1.id_item_61.unique()
df1.shape

1aiSfPuDDbTE9D9wtrq5GEFgl9jXEZoeM


(156, 23)

In [23]:
# NUTRITIONAL VALUE PART 2

#166, 167, 217,238, 283, 284, 290, 291, 292, 296 --> these are the nutritional values that have been repeated and idk how to work with it so will come back to it later
#df2.drop(['166', '167', '217', '238', '283', '284', '290', '291', '292', '296'])

df2 = df1.reset_index(drop=True).set_index('id_item_61')
df2.index = df2.index.astype(str)

#df2 = df2.loc[~df2.index.duplicated(),:].iloc[:,4:].fillna(0)
df2 = df2.loc[~df2.index.duplicated(),:].fillna(0)
df2.shape


(147, 22)

In [24]:
# NUTRITIONAL VALUE PART 3

# extracting only the items that have the units of kg, l, and g
df3 = (df2.loc[df2['unit'].isin(['kg','litre', 'g'])])
df3.columns

# converting the units 
#kg --> kg, g --> kg (*0.001), l --> kg(*1)
# we want everything to be in kgs because then we can find the differences in nutritional value in kg 

cols = ['protein', 'fat', 'fibre',
       'carbohydrate', 'energy_kcal', 'energy_kj', 'calcium', 'iron',
       'betacarotene', 'caroten_total', 'thiamine', 'riboflavin', 'niacin',
       'ascorbic_total', 'ext_source', 'pu_cal', 'pu_prot', 'pu_fat']

df31 = pd.DataFrame()

for col in cols:
  df31[col] = [df3[col][i]/1000 if df3['unit'][i] =='g' else df3[col][i] for i in df3.index]

df31.index

RangeIndex(start=0, stop=111, step=1)

In [25]:
# NUTRITIONAL VALUE PART 4

#adding the index values to the new df4
df31.columns.name = "id_item_68"
df31.index = (['101','102','103','104','105','106','107','108','110','111','112','113','114', '115','116','117','118','120','121','122','139','140','141','142',
 '143', '144', '145', '146', '147', '148', '150', '151', '152', '153', '160', '161', '162', '163', '164', '165', '170', '171', '172', '173', '174', '181','182',
 '183','184', '185', '186', '190', '191', '192', '193', '194', '195', '196', '197', '198', '200', '201', '202', '203', '204', '205', '206', '207', '208', '210',
 '211', '212', '213', '214', '215', '216', '217', '218','220','231','232','235', '236','238','240','241','242','243','244','245','246','250','251','252',
 '253', '254', '255', '256', '257', '260', '261', '262', '263', '264', '295', '296', '304', '331', '332', '333', '334'])

df31.head()

id_item_68,protein,fat,fibre,carbohydrate,energy_kcal,energy_kj,calcium,iron,betacarotene,caroten_total,thiamine,riboflavin,niacin,ascorbic_total,ext_source,pu_cal,pu_prot,pu_fat
101,78.099998,5.500000,37.400002,771.600037,3513.423096,14710.0,81.099998,7.200000,0.0,469.000000,1.7,0.6,25.100000,0.0,0.0,3460.0,75.0,5.0
102,78.099998,5.500000,37.400002,771.600037,3513.423096,14710.0,81.099998,7.200000,0.0,469.000000,1.7,0.6,25.100000,0.0,0.0,3460.0,75.0,5.0
103,74.400002,11.400000,34.599998,767.500000,3534.919189,14800.0,91.899994,44.599998,0.0,336.100006,1.2,0.4,16.000000,0.0,0.0,3460.0,66.0,12.0
104,74.699997,16.200001,25.599998,776.799988,3534.919189,14800.0,150.899994,45.500000,0.0,504.599976,1.1,0.4,18.700001,0.0,0.0,3250.0,75.0,1.0
105,74.699997,16.200001,25.599998,776.799988,3534.919189,14800.0,150.899994,45.500000,0.0,504.599976,1.1,0.4,18.700001,0.0,0.0,3250.0,75.0,1.0


# **Food Expenditure Data**

In [26]:
# FOOD EXPENDITURE DATA PART 1

# here we list the files within the Zipfile
with ZipFile('/content/gdrive/My Drive/NSSOData/Nss62_1.0_new format.zip', 'r') as zip:
  for fn in zip.namelist():
    print(fn)

Nss62_1.0_new format/autorun.inf
Nss62_1.0_new format/block 10_ monthly household expenditure on misc goods and services.dta
Nss62_1.0_new format/AutorunPro.exe
Nss62_1.0_new format/block 10_ monthly household expenditure on misc goods and services.sas7bdat
Nss62_1.0_new format/Block 10_ Monthly household expenditure on misc goods and services.sav
Nss62_1.0_new format/block 11_household expenditure on durables.dta
Nss62_1.0_new format/block 11_household expenditure on durables.sas7bdat
Nss62_1.0_new format/Block 11_Household expenditure on durables.sav
Nss62_1.0_new format/block 3_household characteristics.sas7bdat
Nss62_1.0_new format/block 3_household characteristics.dta
Nss62_1.0_new format/Block 3_Household Characteristics.sav
Nss62_1.0_new format/block 4_person records.dta
Nss62_1.0_new format/block 4_person records.sas7bdat
Nss62_1.0_new format/Block 4_Person records.sav
Nss62_1.0_new format/block 5_monthly household expenditure on food and non food items.dta
Nss62_1.0_new format

In [0]:
# FOOD EXPENDITURE DATA PART 2

# extract only the food expenditure file that we need to analyze from a zipped file 
with ZipFile('/content/gdrive/My Drive/NSSOData/Nss62_1.0_new format.zip', 'r') as zip:
  #for fn in zip.namelist():
    #print(fn)
  zip.extract("Nss62_1.0_new format/block 5_monthly household expenditure on food and non food items.dta","./")
  

In [28]:
# FOOD EXPENDITURE DATA PART 3

# here we open the consumption dta file in stata format

import pandas as pd
df4 = pd.read_stata("Nss62_1.0_new format/block 5_monthly household expenditure on food and non food items.dta")

df4.columns

Index(['HHID2', 'CentreCodeRoundShift', 'Vill_Blk_Slno', 'Round',
       'ScheduleNumber', 'Sample', 'Sector', 'St_Region', 'State', 'District',
       'Stratum', 'SubStratum', 'SubRound', 'SubSample', 'FODSubRegion',
       'SegmentNo', 'Stage2_Stratum', 'Hhold_no', 'Level', 'B5_q1', 'B5_q3',
       'B5_q4', 'B5_q5', 'nss', 'nsc', 'mlt', 'WGT_SS', 'WGT_SS_Combined'],
      dtype='object')

In [0]:
# FOOD EXPENDITURE DATA PART 4

# here we extract the data from UP because it has a max difference in Control vs Treatment groups of 3 districts!

df41 = df4.query("State =='09'")
df41.columns

#a = df5['District'].unique()
df42 = df41.filter(['HHID2', 'District'])
df42 = df42.fillna(0)



#**Food Expenditure Calculation:**

In [40]:
UPfood = df41[['HHID2','B5_q1','B5_q4']].set_index(['HHID2','B5_q1']).squeeze().unstack('B5_q1')

#once again dropped the values that repeat
UPfood.drop(columns = ['166', '167', '217', '238', '283', '284', '290', '291', '292', '296'])
UPfood = UPfood.fillna(0)
UPfood.columns.name = "id_item_68"
UPfood.head()
UPfood['Total Price'] = UPfood.sum(axis = 1)
UPfood

#UPfood2 = UPfood.loc['290']


id_item_68,101,102,103,104,105,106,107,108,110,111,...,326,327,329,330,331,332,333,334,339,Total Price
HHID2,,,,,,,,,,,,,,,,,,,,,
250001101,0.0,100.0,0.0,0.0,0.0,0.0,0.0,588.00,0.0,12.0,...,0.0,200.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,5008.0
250001201,0.0,70.0,28.0,8.0,7.0,0.0,0.0,292.00,6.0,6.0,...,0.0,150.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,3734.0
250002101,0.0,480.0,70.0,80.0,70.0,0.0,0.0,260.00,6.0,6.0,...,0.0,60.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,4421.0
250002201,0.0,280.0,10.0,30.0,25.0,0.0,0.0,780.00,24.0,12.0,...,0.0,150.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,9505.0
250011101,15.0,0.0,0.0,0.0,0.0,0.0,40.0,210.00,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1035.5
250011102,0.0,39.0,0.0,0.0,0.0,0.0,0.0,420.00,0.0,0.0,...,0.0,60.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,3612.0
250011201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3030.0
250011202,0.0,45.0,0.0,0.0,0.0,0.0,0.0,420.00,0.0,0.0,...,0.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,4480.0
250021101,0.0,720.0,0.0,0.0,0.0,0.0,0.0,180.00,0.0,10.0,...,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,8301.9


In [41]:
price = UPfood.merge(df42, left_on = 'HHID2', right_on = 'HHID2').drop_duplicates()
price = price.filter(['HHID2','District', 'Total Price'])
price


,HHID2,District,Total Price
0,250001101,36,5008.0
51,250001201,36,3734.0
105,250002101,36,4421.0
158,250002201,36,9505.0
218,250011101,35,1035.5
255,250011102,35,3612.0
307,250011201,35,3030.0
319,250011202,35,4480.0
367,250021101,53,8301.9
419,250021102,53,19558.0


In [0]:
UPfood.mean()*1000

In [42]:
#Uploading the district name and phase information

!pip install --upgrade -q gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('UP District and Code').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
UProws = pd.DataFrame.from_records(rows)

UProws.columns = ['District', 'District Name (68)', 'MGNREGA Phase']
UProws.head()



[['23', 'Kheri', '1'], ['24', 'Sitapur', '1'], ['25', 'Hardoi', '1'], ['26', 'Unnao', '1'], ['28', 'Rae Bareli ', '1'], ['35', 'Jalaun', '1'], ['37', 'Lalitpur', '1'], ['38', 'Hamirpur', '1'], ['39', 'Mahoba', '1'], ['40', 'Banda', '1'], ['41', 'Chitrakoot', '1'], ['42', 'Fatehpur', '1'], ['43', 'Pratapgarh', '1'], ['44', 'Kaushambi', '1'], ['46', 'Barabanki', '1'], ['58', 'Gorakhpur', '1'], ['59', 'Kushinagar', '1'], ['61', 'Azamgarh', '1'], ['64', 'Jaunpur', '1'], ['66', 'Chandauli', '1'], ['69', 'Mirzapur', '1'], ['70', 'Sonbhadra', '1'], ['17', 'Etah', '2'], ['19', 'Budaun', '2'], ['29', 'Farrukhabad', '2'], ['33', 'Kanpur Dehat ', '2'], ['36', 'Jhansi', '2'], ['48', 'Ambedkar Nag', '2'], ['49', 'Sultanpur', '2'], ['50', 'Bahraich', '2'], ['51', 'Shrawasti', '2'], ['52', 'Balrampur', '2'], ['53', 'Gonda', '2'], ['54', 'Siddharthnagar', '2'], ['55', 'Basti', '2'], ['56', 'S. Kabir Nagar ', '2'], ['57', 'Maharajganj', '2'], ['62', 'Mau', '2'], ['63', 'Ballia', '2'], ['1', 'Saharanpur

,District,District Name (68),MGNREGA Phase
0,23,Kheri,1
1,24,Sitapur,1
2,25,Hardoi,1
3,26,Unnao,1
4,28,Rae Bareli,1


In [43]:
# mean of price

meanprice62 = UProws.merge(price, left_on ='District', right_on = 'District').drop_duplicates()
meanprice62.head()
meanprice62.groupby('MGNREGA Phase')['Total Price'].mean()

MGNREGA Phase
1    4363.924580
2    4435.562224
3    4381.212688
Name: Total Price, dtype: float64

In [44]:
# sd of price

sdprice62 = UProws.merge(price, left_on ='District', right_on = 'District').drop_duplicates()
sdprice62.head()
sdprice62.groupby('MGNREGA Phase')['Total Price'].std()

MGNREGA Phase
1    3154.647742
2    2631.369909
3    2572.214744
Name: Total Price, dtype: float64

In [45]:
# count in each phase

df43 = df42.groupby('District').count()
count = UProws.merge(df43, left_on ='District', right_on = 'District').drop_duplicates()
count62 = count.groupby('MGNREGA Phase')['HHID2'].sum()
count62

MGNREGA Phase
1    40597
2    31138
3    72354
Name: HHID2, dtype: int64

# **Nutritional Value Intake:**

In [50]:
UPfood1 = df41[['HHID2','B5_q1','B5_q3']].set_index(['HHID2','B5_q1']).squeeze().unstack('B5_q1')

#once again dropped the values that repeat
UPfood1.drop(columns = ['166', '167', '217', '238', '283', '284', '290', '291', '292', '296'])
UPfood1 = UPfood1.fillna(0)
UPfood1.columns.name = "id_item_68"
UPfood1['Total Price'] = UPfood1.sum(axis = 1)
UPfood1

#UPfood2 = UPfood.loc['290']

id_item_68,101,102,103,104,105,106,107,108,110,111,...,326,327,329,330,331,332,333,334,339,Total Price
HHID2,,,,,,,,,,,,,,,,,,,,,
250001101,0.0,10.0,0.00,0.00,0.00,0.0,0.0,84.0,0.0,1.00,...,0.0,600.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,3166.25
250001201,0.0,5.0,1.00,0.25,0.25,0.0,0.0,45.0,0.5,0.50,...,0.0,450.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,2507.85
250002101,0.0,4.0,0.25,0.25,0.25,0.0,0.0,40.0,0.5,0.50,...,0.0,180.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,1931.95
250002201,0.0,20.0,0.50,1.00,1.00,0.0,0.0,120.0,2.0,1.00,...,0.0,450.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,4275.15
250011101,5.0,0.0,0.00,0.00,0.00,0.0,15.0,30.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,1589.25
250011102,0.0,5.0,0.00,0.00,0.00,0.0,0.0,60.0,0.0,0.00,...,0.0,300.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,3192.55
250011201,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,178.50
250011202,0.0,5.0,0.00,0.00,0.00,0.0,0.0,60.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,3354.25
250021101,0.0,60.0,0.00,0.00,0.00,0.0,0.0,22.5,0.0,1.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,4701.00


In [49]:
use = list(set(df31.index.tolist()).intersection(UPfood1.columns))
N = UPfood1[use].dot(df31.loc[use,:])
N.head()
N.columns

N.head()

id_item_68,protein,fat,fibre,carbohydrate,energy_kcal,energy_kj,calcium,iron,betacarotene,caroten_total,thiamine,riboflavin,niacin,ascorbic_total,ext_source,pu_cal,pu_prot,pu_fat
HHID2,,,,,,,,,,,,,,,,,,
250001101,12481.522634,6564.411941,11255.861705,74611.772529,407213.799280,1.704923e+06,66161.884283,4272.384031,60325.244486,548250.478195,437.400699,168.271219,2840.494638,5155.005798,0.0,432558.0,13799.80,5993.70
250001201,7503.576166,7419.494854,6378.269729,43576.233379,269333.579980,1.127646e+06,65828.210417,2541.960304,71901.736300,512433.845497,246.531811,118.523049,1528.491584,4979.754581,0.0,290583.0,8358.80,7510.95
250002101,7270.925396,6904.888258,5622.198168,39083.446748,245899.555801,1.029532e+06,80178.460210,2254.885063,61416.817857,449835.483839,224.265593,124.262684,1350.642192,5301.014850,0.0,269469.0,8166.90,7453.90
250002201,20335.681258,18874.254808,16421.204527,118472.535323,722907.081276,3.026667e+06,180595.985512,6472.385196,87667.836115,926913.086487,657.436781,315.678050,4098.436586,8265.939589,0.0,778527.5,22714.05,19689.95
250011101,5754.482323,1992.029756,5956.679621,35959.968529,187596.513922,7.854291e+05,23911.970869,2125.010663,95897.572658,515991.558560,223.653641,80.946097,1405.403171,4837.069160,0.0,197365.0,6437.60,1911.80


In [52]:
P = df41.filter(['HHID2', 'District'])
Q = N.merge(P, left_on ='HHID2', right_on = 'HHID2').drop_duplicates()
Q.head() 

R = Q.groupby('District').mean()
R

S = R.merge(UProws, left_on = 'District', right_on = 'District').drop_duplicates()
S.head()

,District,protein,fat,fibre,carbohydrate,energy_kcal,energy_kj,calcium,iron,betacarotene,...,thiamine,riboflavin,niacin,ascorbic_total,ext_source,pu_cal,pu_prot,pu_fat,District Name (68),MGNREGA Phase
0,10,5704.911800,5218.119563,4589.303344,33040.269241,199191.121318,8.339734e+05,56080.520361,1888.827400,119056.402218,...,176.851414,130.168911,1232.216797,7216.986771,0.355208,219377.791667,6328.975000,5187.946875,G. Buddha Nagar,3
1,11,7981.894595,20133.674227,6214.953163,43671.641861,382994.036656,1.603520e+06,84911.337995,2541.810517,92509.787163,...,242.316260,130.747660,1576.699298,8457.640749,0.228169,411011.387042,8918.503535,20191.782394,Bulandshahr,3
2,12,7539.206172,7493.846447,5824.820670,40575.582398,255594.443027,1.070123e+06,80569.569579,2389.641097,111041.048692,...,224.308286,124.821132,1481.791958,10845.790496,0.240000,281398.766250,8398.117875,7581.974375,Aligarh,3
3,13,8381.348183,7499.561219,6350.508789,44594.673726,275501.389874,1.153469e+06,90991.361927,2601.270005,73887.841737,...,252.399107,139.844921,1593.408998,6950.575615,0.000000,303545.037500,9366.512188,7828.203125,Hathras,3
4,14,9234.771919,7714.585551,7786.286503,52416.247216,312036.509534,1.306434e+06,90711.258878,3100.114285,110909.251769,...,303.478934,152.315563,1868.677416,8386.176797,2.132812,348660.242188,11069.835156,7837.250781,Mathura,3


In [54]:
#mean
mean62 = S.groupby('MGNREGA Phase').mean()
mean62


,protein,fat,fibre,carbohydrate,energy_kcal,energy_kj,calcium,iron,betacarotene,caroten_total,thiamine,riboflavin,niacin,ascorbic_total,ext_source,pu_cal,pu_prot,pu_fat
MGNREGA Phase,,,,,,,,,,,,,,,,,,
1,10172.105568,6773.942674,7926.605256,62817.083992,352778.964228,1.477015e+06,73690.658845,2905.987915,113920.434691,664655.442020,308.093957,145.909057,2316.679146,9779.514285,0.330845,375995.414476,11008.303484,6790.559877
2,11346.054017,8919.024636,8421.122682,65822.598110,388028.335977,1.624597e+06,79928.653499,3091.669599,104847.268088,670157.938775,329.329529,148.516796,2753.325099,10323.523218,0.227990,413813.955628,12181.953961,8890.419515
3,9219.067966,8028.446119,7213.466077,54196.014799,324221.605795,1.357451e+06,79159.702057,2744.411372,120361.638588,663376.186036,277.847866,140.626748,1988.585216,10102.385174,0.623293,350206.977599,10154.032741,8129.718819


In [55]:
#std
std62 = S.groupby('MGNREGA Phase').std()
std62

,protein,fat,fibre,carbohydrate,energy_kcal,energy_kj,calcium,iron,betacarotene,caroten_total,thiamine,riboflavin,niacin,ascorbic_total,ext_source,pu_cal,pu_prot,pu_fat
MGNREGA Phase,,,,,,,,,,,,,,,,,,
1,1863.547767,1064.695369,1452.535431,12119.828295,62277.949806,260745.323718,13385.187704,529.604619,47316.004065,215703.864095,55.259805,34.080222,463.322414,3603.965850,0.855673,66017.471480,1950.823308,1034.711042
2,3971.432875,5782.238443,2557.328893,16356.450034,122223.975584,511727.333740,19277.252534,943.774446,52656.625199,172009.005659,97.448399,30.047993,1302.638788,3578.786281,0.204698,130762.540250,4071.522432,5972.679046
3,1709.290055,2890.189787,1377.400230,12001.046620,60111.223124,251673.675289,13242.087007,459.477200,30941.557932,120086.752298,52.316600,21.036493,449.125143,3412.063592,1.609410,61051.628321,1805.950070,2900.401663
